In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import dataProcess

from datetime import datetime

In [9]:
# should use a better way to convert nans to numbers
def nan_to_zero(data):
  for i in range(data.shape[0]):
    for j in range(data.shape[1]):
      for k in range(data.shape[2]):
        if data[i, j, k].isnan:
          data[i, j, k] = 0

In [10]:
# data preprocessing

start = datetime(2024, 1, 3).date()
end = datetime(2024, 4, 11).date()
hourly_path ='weatherstats_toronto_hourly.csv'
daily_path ='weatherstats_toronto_daily.csv'

# data
data_tensor_preprocess = dataProcess.dataToTensorHourly(hourly_path, separateByDay=False, missingThreshold=0.1, columnToDelete=['wind_dir', 'unixtime'], start=start, end=end)
print(data_tensor_preprocess[0].shape)
# for some reason going back one day does not work, but this does
# need to go back one day so you're predicting the temperature for the next day
data = data_tensor_preprocess[0].reshape(-1, 24, 13)[:-1]
print(data.shape)
nan_to_zero(data)

# targets
targets = dataProcess.dailyTargets(daily_path, start=datetime(2024, 1, 4).date())
print(targets.shape)

torch.Size([2376, 13])
torch.Size([98, 24, 13])
torch.Size([98])


In [11]:
class WeatherPredictorGRU(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(WeatherPredictorGRU, self).__init__()
    self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
    self.linear = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    out, hidden = self.gru(x)
    last_hidden_state = hidden[-1]
    prediction = self.linear(last_hidden_state)
    return prediction

In [12]:
def process_data_for_rnn_gru(train_fraction, valid_fraction, data, targets, batch_size):

  train_dataset = TensorDataset(data, targets)
  total_size = len(data)
  train_split_point = int(total_size * train_fraction)
  valid_split_point = train_split_point + int(total_size * valid_fraction)

  train_dataset = TensorDataset(data[:train_split_point], targets[:train_split_point])
  val_dataset = TensorDataset(data[train_split_point:valid_split_point],
                              targets[train_split_point:valid_split_point])
  test_dataset = TensorDataset(data[valid_split_point:], targets[valid_split_point:])

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = process_data_for_rnn_gru(0.8, 0.1, data, targets, 1)

In [13]:
# create model
input_size = 13  # number of weather features
hidden_size = 50
output_size = 1  # average temperature
model = WeatherPredictorGRU(input_size, hidden_size, output_size)

In [17]:
learning_rate = 0.001
num_epochs = 10
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

def train_model(model, num_epochs, optimizer, criterion, train_loader, val_loader):

  for inputs, targets in train_loader:
    assert not torch.isnan(inputs).any(), "NaN found in inputs"
    assert not torch.isnan(targets).any(), "NaN found in targets"

  model.train()
  for epoch in range(num_epochs):
    total_loss = 0

    for inputs, targets in train_loader:
      inputs, targets = inputs.float(), targets.float()
      optimizer.zero_grad()
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss / len(train_loader):.4f}')

    model.eval()
    total_val_loss = 0
    with torch.no_grad():
      for inputs, targets in val_loader:
        inputs, targets = inputs.float(), targets.float()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        total_val_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {total_val_loss / len(val_loader):.4f}')

train_model(model, num_epochs, optimizer, criterion, train_loader, val_loader)

Epoch [1/10], Training Loss: 21.4687
Epoch [1/10], Validation Loss: 127.7113
Epoch [2/10], Training Loss: 21.2945
Epoch [2/10], Validation Loss: 126.7376
Epoch [3/10], Training Loss: 21.3012
Epoch [3/10], Validation Loss: 126.1369
Epoch [4/10], Training Loss: 21.3068
Epoch [4/10], Validation Loss: 125.7649
Epoch [5/10], Training Loss: 21.3102
Epoch [5/10], Validation Loss: 125.4933
Epoch [6/10], Training Loss: 21.3140
Epoch [6/10], Validation Loss: 125.3842
Epoch [7/10], Training Loss: 21.3151
Epoch [7/10], Validation Loss: 125.3806
Epoch [8/10], Training Loss: 21.3112
Epoch [8/10], Validation Loss: 125.3392
Epoch [9/10], Training Loss: 21.3107
Epoch [9/10], Validation Loss: 125.3092
Epoch [10/10], Training Loss: 21.3105
Epoch [10/10], Validation Loss: 125.2953
